In [1]:
import sys
import os
from tensorflow import Graph, Session
import numpy as np
from sklearn import preprocessing
from skimage.transform import resize
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TKAgg', warn=False)
from skimage import color
from math import log
import win_unicode_console
win_unicode_console.enable()

from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, Lambda, Layer, Conv2D, MaxPooling2D, UpSampling2D
from keras import losses
from keras.callbacks import LambdaCallback, EarlyStopping
import keras.backend as K
import stockdatautils as datautils
import pstStockVAE
import ftrStockVAE

C:\Users\lukes\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\Users\lukes\Anaconda3\lib\site-packages\win_unicode_console\__init__.py:31: RuntimeWarning: sys.stdin.encoding == 'cp1252', whereas sys.stdout.encoding == 'UTF-8', readline hook consumer may assume they are the same
  readline_hook.enable(use_pyreadline=use_pyreadline)
Using TensorFlow backend.


In [6]:
def gen_embeds(stocks, modes, model, ext, dir):
    for s in stocks:
        for m in modes:
            directory = os.path.abspath(dir + s + '/' + s + '_' + m)
            fnames = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.ftr.npy')]

            for fname in fnames:
                x = np.load(fname)
                x = np.reshape(x, newshape=(1,x.shape[0],x.shape[1],x.shape[2]))
                x_embed = model.embed(x)[0]
                np.save(fname[:-4] + ".emb", x_embed)

In [7]:
history = 20
forecast = 5
width = 128
height = width
stocks = ["MMM", "GE"]
modes = ["train", "test"]

pstVAE = pstStockVAE.pstStockVAE(optimizer='adagrad')
pstVAE.load_weights('saved_model/pstStockVAE')

gen_embeds(stocks, modes, pstVAE, '.pst.npy', '../Data/')

ftrVAE = ftrStockVAE.ftrStockVAE(optimizer='adagrad')
ftrVAE.load_weights('saved_model/ftrStockVAE')

gen_embeds(stocks, modes, ftrVAE, '.ftr.npy', '../Data/')

Stock VAE
--------------------

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 16) 160         encoder_input[0][0]              
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 64, 64, 16)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 64, 64, 16)   2320        max_pooling2d_1[0][0]            
_____________________________________________________________________________